In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.1264298027434827,0.8391548409332943,0.36512812…
  RigidBodyDynamics.Joint… => [0.7245848806385393]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [9]:
cache = MechanismStateCache(mechanism, x);

RigidBodyDynamics.CartesianFrame3D("world")
RigidBodyDynamics.CartesianFrame3D("after_1")


In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),0.0794334279061466 - 0.2691606714190625im + 0.3052328534349393jm - 0.909986685988473km,Vec(0.19683924243192563,0.8747508668616453,0.7414285020119662))

In [11]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-1.8602064628241193,0.3092984319337856,0.44655697254664334))

In [12]:
mass(mechanism)

0.5051188932027741

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-1.4289549910292134,0.5782404025072473,0.9161606386372684))

In [14]:
relative_twist(cache, body2, body1)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(-0.5199810183094736,0.2758876105927913,-0.4441376242552466),Vec(-0.509575688130768,-1.1110388166970566,-0.09355694479629484))

In [15]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.0,0.0,0.0),Vec(0.0,0.0,0.0))

In [16]:
geometric_jacobian(cache, path(mechanism, body2, world))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
  0.717626   0.39731    -0.0511316  …  -0.0       -0.0        -0.0       
 -0.380753  -0.121982   -0.992529      -0.0       -0.0        -0.0       
  0.612955  -0.909541    0.110777      -0.0       -0.0        -0.0       
  0.703266  -0.365187   -0.0900324      0.39731   -0.0511316   0.916259  
  1.53335   -0.68761     0.084113      -0.121982  -0.992529   -0.00249354
  0.129118  -0.0673041   0.712072   …  -0.909541   0.110777    0.400579  )

In [17]:
mass_matrix(cache)

7x7 Array{Float64,2}:
  0.10848    1.43989     0.135034  …  -0.213321  -0.41122       0.638136
 -1.69432    0.586929    1.52483       1.15269   -0.056551      0.616093
  0.532255  -1.0945     -0.54672      -0.275064   0.651777     -0.386361
  1.66253   -0.821756   -1.24892      -0.764383   0.081522     -0.928609
 -0.16416    1.51941     0.480372      0.0       -0.592801      0.685703
 -0.61791    0.0833643   0.826603  …   0.592801   6.93889e-18   0.183761
  0.638136   0.616093   -0.386361      0.685703   0.183761     -0.907926

In [18]:
 momentum_matrix(cache).mat

6x7 Array{Float64,2}:
  0.0696817   0.284693   -0.0143283  -0.199599   …  -0.460307    -0.000639145
 -0.41303     0.0495068   0.49949     0.0612811      0.00125269   0.00179166 
  0.0858319   0.574973    0.0358829   0.456932      -0.201241     0.00186122 
 -0.102769    0.327799   -0.236612    0.113779      -0.0627004    0.0446513  
  0.0383259   1.40803     0.326498    0.758469      -0.578428    -0.0492724  
  1.77785    -0.949131   -1.52028    -0.0520199  …   0.139816    -0.0298309  